In [ ]:
import mlflow
import mlflow.pytorch

In [ ]:
mlrun = './mlruns/0/'

In [ ]:
! ls ./mlruns/0

In [ ]:
import yaml
import os
import glob

In [ ]:
with open(os.path.join(mlrun, 'meta.yaml'), 'r') as f:
    meta = yaml.safe_load(f)

In [ ]:
meta

In [ ]:
glob.glob(os.path.join(mlrun, '*'))

In [ ]:
rundir = './mlruns/0/f63a82ab86524a7c8708310d61c54431/'

In [ ]:
glob.glob(rundir)

In [ ]:
with open(os.path.join(rundir, 'meta.yaml'), 'r') as f:
    runmeta = yaml.safe_load(f)

In [ ]:
runmeta

In [ ]:
! ls ./mlruns/0/f63a82ab86524a7c8708310d61c54431/artifacts/path/data/

In [ ]:
def read_params(run_path: str):
    params = dict()
    for paramName in glob.glob(os.path.join(run_path, 'params/*')):
        print(paramName)
        with open(paramName, 'r') as f:
            value = f.read()
        params[os.path.basename(paramName)]=value
    return params

In [ ]:
read_params(run_path='./mlruns/0/f63a82ab86524a7c8708310d61c54431')

## using client

In [ ]:
# get_run(run_id: str) 
from mlflow.client import MlflowClient

In [142]:
import mlflow.pytorch

import mlflow.onnx

In [ ]:
local_client = MlflowClient()

In [85]:
experiment = local_client.get_experiment(experiment_id='0')

In [124]:
runs = local_client.search_runs(experiment_ids=[experiment.experiment_id]) #, order_by='start_time DESC'

In [125]:
for run in runs:
    print(run.info.run_id)

e10f5cc7f8c44f079fd8214d4f14b840
7d8add9e6d6b424e9aef34c3a8b84c56
fba2fecd08e4444098a0a9b81c8759d7
21fe09e3049a4ef491b32124041ba10d
f63a82ab86524a7c8708310d61c54431
943e0331574c496a8d4133039c547f57
faad19c43a20438ba1d392957ac870be
6f85e591e09a42dcbc225539359fca23
a4217081db4d41c9b4c2deb2844f44a1
481dcf3917214c4e8b2d5ff94244d168
5eca6cb9aa0f452a8991f444a8dccc8f
ae450efcf5114dc1b2fecbc129107c4b
ea63cb7ec24d4ca0a63c3094dfa5c5e7
2b2ab36e96c0465f8e00362e0d3f8d2e
1e6c9f06033343a79b6c26c397fb3cba
7f2fd88ec2f9435faca1bcc0e803ac92
489269e357cc4ecb8a9411b2b9310f31
e6a42d424a7640daac859de008c62490
3d787a300d6a4cf7bd5bfee483baf2ba


In [126]:
r = runs[0]

In [115]:
# client.log_batch

In [129]:
r.info.run_id

'e10f5cc7f8c44f079fd8214d4f14b840'

In [131]:
r.data.params

{}

In [132]:
r.info.artifact_uri

'file:///home/jj/git/modelrepository/notebooks/mlruns/0/e10f5cc7f8c44f079fd8214d4f14b840/artifacts'

In [135]:
! ls /home/jj/git/modelrepository/notebooks/mlruns/0/e10f5cc7f8c44f079fd8214d4f14b840/artifacts/

conda.yaml  data  MLmodel  python_env.yaml  requirements.txt


In [89]:
# this does not work
#local_client.get_registered_model(name='suave-cat-581')
#local_client.search_registered_models()

In [158]:
r.par

<Run: data=<RunData: metrics={}, params={}, tags={'mlflow.log-model.history': '[{"run_id": "e10f5cc7f8c44f079fd8214d4f14b840", '
                             '"artifact_path": "", "utc_time_created": '
                             '"2022-11-02 10:03:15.314602", "flavors": '
                             '{"pytorch": {"model_data": "data", '
                             '"pytorch_version": "1.11.0+cu102", "code": '
                             'null}, "python_function": {"pickle_module_name": '
                             '"mlflow.pytorch.pickle_module", "loader_module": '
                             '"mlflow.pytorch", "python_version": "3.10.6", '
                             '"data": "data", "env": "conda.yaml"}}, '
                             '"model_uuid": '
                             '"f4cc3196224b482ca49f503c49485c98", '
                             '"mlflow_version": "1.30.0"}]',
 'mlflow.runName': 'burly-turtle-743',
 'mlflow.source.name': '/home/jj/anaconda3/envs/wheat/lib/

In [139]:
a_model = mlflow.pytorch.load_model(r.info.artifact_uri)

In [140]:
a_model

SimpleNet(
  (l1): Linear(in_features=784, out_features=10, bias=True)
)

In [161]:
runs = local_client.search_runs(experiment_ids=[experiment.experiment_id])

for run in runs:
    models = []
    print('Processing run:', run.info.run_id)
    
    
    for metric in run.data.metrics:
        metric_history = local_client.get_metric_history(run_id=run.info.run_id, key=metric)
        print(f"Got metric history for {metric} length={len(metric_history)}")
    
    print('Params:', run.data.params)
    
    artifacts = local_client.list_artifacts(run_id=run.info.run_id)
    models = [run.info.artifact_uri for art in artifacts if art.path.startswith('MLmodel')]
    if any(models):
        a_model = mlflow.pytorch.load_model(models[0])
        print('Got model path', models, a_model)
        
    
    print('-'*10)

Processing run: e10f5cc7f8c44f079fd8214d4f14b840
Params: {}
Got model path ['file:///home/jj/git/modelrepository/notebooks/mlruns/0/e10f5cc7f8c44f079fd8214d4f14b840/artifacts'] SimpleNet(
  (l1): Linear(in_features=784, out_features=10, bias=True)
)
----------
Processing run: 7d8add9e6d6b424e9aef34c3a8b84c56
Got metric history for train_loss length=60000
Params: {'param1': 'torch.Size([10])', 'param0': 'torch.Size([10, 784])'}
----------
Processing run: fba2fecd08e4444098a0a9b81c8759d7
Got metric history for train_loss length=60000
Params: {'param1': 'torch.Size([10])', 'param0': 'torch.Size([10, 784])'}
----------
Processing run: 21fe09e3049a4ef491b32124041ba10d
Params: {'param1': 'torch.Size([10])', 'param0': 'torch.Size([10, 784])'}
----------
Processing run: f63a82ab86524a7c8708310d61c54431
Got metric history for train_loss length=60000
Params: {'param1': 'torch.Size([10])', 'param0': 'torch.Size([10, 784])'}
----------
Processing run: 943e0331574c496a8d4133039c547f57
Got metric hi